# XJCO2121 Data Mining - Coursework 2

Author: Tang Shiyu

Student ID: 2022116003

In [85]:
import pandas as pd
import chardet

### This file is for data collecting and processing

For AI-advertisement theme:

In [87]:
files = ['Advertising Planning', 'Additional', 'Brand Marketing']
dfs = []

for file in files:
    # Find the encoding of the file
    with open(f'data/ai-Advertisement/Comments-{file}.csv', 'rb') as f:
        result = chardet.detect(f.read())
    # Read the data
    df = pd.read_csv(f'data/ai-Advertisement/Comments-{file}.csv', encoding=result['encoding'], encoding_errors='replace')
    # Check the data
    print(f'The number of data in {file} is {len(df)}')
    # Delete the unneeded columns
    df = df.drop(columns=['ip_location', 'note_id', 'user_id', 'avatar', 'pictures', 'parent_comment_id', 'last_modify_ts'])
    dfs.append(df)
# Merging all data in this theme
df_ad = pd.concat(dfs)
# Delete the data with the same comment_id
df_ad = df_ad.drop_duplicates(subset=['comment_id'])
print(f'After deleting repeated data, the number of data is {len(df_ad)}')
df_ad.head()

The number of data in Advertising Planning is 7514
The number of data in Additional is 7744
The number of data in Brand Marketing is 3739
After deleting repeated data, the number of data is 10539


,comment_id,create_time,content,nickname,sub_comment_count,like_count,sentiment
0,667408e60000000017012a2a,1718880488000,还好我boss不懂看我出单还表扬我了,木子,131.0,31.0,negative
1,66596c33000000000f0077bc,1717136435000,做的好棒！,豆花要加糖,4.0,2.0,positive
2,665961c1000000000e033c6c,1717133762000,真诚发问这个豆奶口感咋样好喝吗早八人很需要啊宝子,小红薯62E23D58,3.0,2.0,positive
3,667978b100000000150082e2,1719236785000,求分享,捞起月亮的渔民,1.0,0.0,positive
4,665975d0000000001b007630,1717138897000,哇塞你们画的也太多了吧,小红薯63E82F05,3.0,1.0,negative
